In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from keras.models import  Sequential
from keras.layers.core import  Dense, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')
train.info()

## EDA

In [ ]:
train['target'].value_counts()

In [ ]:
train[train.isna()].count()

In [ ]:
def transform_df(df):
    s = df['f_27'].apply(lambda x: [ord(c) - ord('A') for c in x])
    chars = pd.DataFrame.from_dict(dict(zip(s.index, s.values))).T
    df["unique_characters"] = df['f_27'].apply(lambda s: len(set(s)))
    return df.merge(chars, left_index=True, right_index=True).drop('f_27', axis=1)

In [ ]:
transformed_train = transform_df(train)

In [ ]:
scaler = StandardScaler()
scaled_train = scaler.fit_transform(transformed_train.drop(['target'], axis=1))

## Model

In [ ]:
model = Sequential([
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['AUC'])

callbacks = [EarlyStopping(patience=20, monitor='val_loss', restore_best_weights=True), 
             ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=20)]

history = model.fit(x=scaled_train, y=train['target'], validation_split=0.2, epochs=200, 
                    callbacks=callbacks, 
                    batch_size=512)

## Submit

In [ ]:
transformed_test = test
transformed_test = transform_df(transformed_test)
scaled_test = scaler.transform(transformed_test)

In [ ]:
preds = model.predict(scaled_test)
test['target'] = preds.reshape((len(test),))

In [ ]:
test.reset_index()[['id', 'target']].to_csv('submit.csv', index=False)